# Автоматизированное и гарантированное обновление пользователя

**Задача** - написать программу, производящую полностью автоматизированное и гарантированное обновление программного комплекса пользователя(ПКП).

Обновление ПКП делится на следующие составляющие:

- **Обновление баз данных**

- **Скачивание оперативных обновлений**

- **Обновление регистрационных файлов**

Скачиваемые файлы (базы данных, оперативные обновления, рег.файлы) находятся на ftp-сервере, к которому каждый клиент имеет доступ с определенными правами. Само скачивание осуществляется по SFTP(SSH File Transfer Protocol)-протоколу. 

Отмечу, что необязательно все составляющие обновления ПКП должны обновляться одновременно. Так, например, может быть ситуация, что нужны только оперативные обновления или обновление регистрационных файлов. В таком случае на ftp-сервере в папке клиента выкладывается флаг соответствующего обновления, сигнализирующий программе о необходимости проведения соответствующего обновления.

После обновления ПКП на ftp-сервер присылается отчет о проведенном ПКП и лог-файл, содержащий важную промежуточную информацию о процессе обновления. Если в отчете обнаружено, что обновление прошло с ошибками, то, как минимум, нужно посмотреть лог-файл.

## 1. Обновление баз данных

**Входные данные:**

- Xml-файл, который содержит список необходимых для скачивания баз данных(файлов) со следующими характеристиками: название файла, объем файла и md5(хеш-сумма) файла.

- Флаг, сигнализирующий о необходимости обновить базы данных

**Выходные данные:**

- По окончании обновления удаляется флаг

- По необходимости (т.е. при наличии флага для отчета) присылается отчет на сервер

**Алгоритм:**

Задача разбивается на три этапа:

1. Отключаем службу Техэксперт, чтобы можно было изменять текущие базы данных в папке Base на ПКП.

2. Скачиваем базы в папку Base. Этот этап разбивается на следующие подэтапы:

    2.1. Скачиваем xml-файл с характеристиками баз данных.
    
    2.2. Проверяем - сколько необходимо место для осуществления скачивания файлов. Если места достаточно, то переходим к пункту 2.3. Иначе - пишем в лог о проблемах с наличием свободного места, и программа заканчивает свою работу. Кроме того, в ходе подсчета свободного места одновременно проверяем - есть ли уже полностью скачанные файлы (т.е. файлы, у которых md5 совпал с md5 из xml). Такая проверка особенно нужна на случай, если программа по тем или иным причинам прекратила свое выполнение и планировщик запустил ее заново. Когда программа запустится заново, то обнаружится, что некоторые файлы уже полностью скачаны, а, значит, эти файлы уже не нужно скачивать. Таким образом экономится время.
    
    2.3. Закачиваем базы данных по xml-файлу. При этом считаем, что файл скачался тогда и только тогда, когда md5 скачанного файл совпал с md5 из xml.
    
    2.4. После скачивания всех файлов делаем заключительную проверку скачанных файлов по md5.

3. Включаем службу

4. Удаляем флаг

## 2. Скачивание оперативных обновлений

**Входные данные:**

- Текстовый файл, содержащий список скачиваемых файлов

- Флаг, сигнализирующий о необходимости скачать оперативные обновления

**Выходные данные:**

- По окончании обновления удаляется флаг

- По необходимости (т.е. при наличии флага для отчета) присылается отчет на сервер

**Алгоритм:**

1. Отключаем службу Техэксперт, чтобы можно было изменять текущие базы данных в папке Base на ПКП.

2. Проходимся по списку скачиваемых файлов и скачиваем файлы. 

3. Включаем службу.

## 3. Обновление регистрационных файлов

**Входные данные:**

- Регистрационный файл

- Флаг, сигнализирующий о необходимости скачать регистрационный файл

**Выходные данные:**

- По окончании обновления удаляется флаг

- По необходимости (т.е. при наличии флага для отчета) присылается отчет на сервер

**Алгоритм:**

1. Отключаем службу Техэксперт, чтобы можно было изменять текущие базы данных в папке Base на ПКП.

2. Перемещаем текущий рег.файл в папку со старыми регами (рег == рег.файл == регистрационный файл)

3. Скачиваем новый рег.файл

4. Включаем службу.

## Подключаем небходимые библиотеки и задаем параметры конфигурации

In [1]:
import sys
import configparser
import os
import logging
import datetime
from dateutil import parser
import psutil
import time
import xml.etree.ElementTree as ET
from lxml import etree
import hashlib
from datetime import datetime, timedelta
import paramiko
from stat import S_ISDIR, S_ISREG
import shutil
import subprocess
import requests

In [2]:
# Отключаем лишние логи, которые возникают из-за использования paramiko.
# https://stackoverflow.com/questions/8144545/turning-off-logging-in-paramiko
logging.getLogger("paramiko").setLevel(logging.WARNING)

In [3]:
# Путь к файлу с конфигами
# Те переменные, которые часто используются в коде и не должны изменять своего значения, я назвал, используя
# заглавные буквы. Например, CUR_DIR
# Чтобы crontab корректно работал, нужно указать полный путь до файла конфигураций.
# https://stackoverflow.com/questions/1432924/python-change-the-scripts-working-directory-to-the-scripts-own-directory

# https://stackoverflow.com/questions/39125532/file-does-not-exist-in-jupyter-notebook
abspath = os.path.abspath(__file__)
CUR_DIR = os.path.dirname(abspath)
#CUR_DIR = os.path.abspath('')
os.chdir(CUR_DIR)

PATH_TO_CONFIG = 'Config.ini'

# Считываем конфигурации из файла Config_lin.ini
read_config = configparser.ConfigParser()
read_config.read(PATH_TO_CONFIG)

# Имя клиента
CLIENT_NAME = read_config.get('Information', 'client')

# Внешний ip-адрес ftp-сервера, с которого производится скачивание файлов
HOST = '62.183.112.196'

# Порт, по которому подключаемся по sftp
PORT = 17523

# Логин и пароль для пользователя
USER = read_config.get('Information', 'user')
PASSWD = read_config.get('Information', 'passwd')

# Папка на сервере, где хранятся списки скачиваемых файлов и их характеристики
SERVER_BASE_FOLDER = read_config.get('Information', 'server_base_folder')

# Путь к папке, где установлен техэксперт
PATH_TO_TEXPERT = read_config.get('Information', 'texpert')

# Путь к папке на локальном компьютере пользователя, куда нужно скачивать файлы
PATH_TO_DOWNLOAD = os.path.join(PATH_TO_TEXPERT, "Base")

# Путь к папке, где хранится кеш программы
PATH_TO_CACHE = os.path.join(PATH_TO_TEXPERT, "cache")

# Название скачиваемого xml файла
XML_NAME = read_config.get('Information', 'xml_name')

# Путь к папке на серверной части, где хранятся скачиваемые файлы *.ud6
PATH_TO_SERVER_OPERUP = read_config.get('Information', 'server_operup_folder')

# Путь к файлу серверной части, где хранится список скачиваемых файлов
PATH_TO_SERVER_OPERUP_LIST = '/' + CLIENT_NAME.upper() + '/FILESLISTS/OperupList.txt'

# Путь к папке на серверной части, куда нужно отправлять логи
PATH_TO_SERVER_LOGS = '/' + CLIENT_NAME.upper() + '/LOGS/'

# Путь к папке, куда нужно скачать оперативные обновления (*.ud6)
PATH_TO_OPERUP_DESTINATION = os.path.join(PATH_TO_TEXPERT, "Operup")

# Путь к скачиваемому xml файлу
XML_PATH = '/' + CLIENT_NAME.upper() + '/FILESLISTS/'

# Папка, куда мы скачиваем файл-отчет
FILE_REPORT_DESTINATION='/' + CLIENT_NAME.upper() + '/SI/'

# Адрес, откуда скачиваем файл-отчет
PORT_FOR_REPORT = read_config.get('Information', 'port_for_report')
FILE_REPORT_URL = 'http://127.0.0.1:{}/sysinfo/si_save_request'.format(PORT_FOR_REPORT)

# Путь к папке FlagsList
PATH_TO_FLAG = '/' + CLIENT_NAME.upper() + '/FLAGS'

# Путь к файлу с логами
PATH_TO_LOG = read_config.get('Information', 'logs')

In [4]:
# Функция, находящая hash файла
# https://stackoverflow.com/questions/3431825/generating-an-md5-checksum-of-a-file
def md5(fname):
    hash_md5 = hashlib.md5()
    with open(fname, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

In [5]:
retry = True

while (retry):
    
    try:
        transport = paramiko.Transport((HOST,PORT))
        
        # Авторизуемся в созданном канале 
        transport.connect(None,USER,PASSWD)

        # Инициализируем sftp
        sftp = paramiko.SFTPClient.from_transport(transport)

        retry = False
    
    except paramiko.ssh_exception.SSHException as e:
        retry = True

### 1. Проверка

В случае, если скрипт был запущен заново (было отключение света или еще что-то), то нужно понять - что нужно сейчас делать? Для этого используется класс **Checker**, который проверяет текущее состояние дел. Например, нужно ли скачивать стандартные обновления?

Введем понятие **модуль**. Под **модулем** стоит понимать:

- Обновление баз данных (стандартное обновление)

- Обновление регистрационного файла

- Оперативные обновления

- Очистка кеша

- Отправка отчета

Каждый модуль характеризуется своим флагом, который выложен на ftp-сервере. Если этот флаг есть - значит, соответствующий модуль нужно сделать. Иначе - модуль делать не нужно. По окончании каждого модуля соответствующий флаг удаляется. 

In [6]:
class Checker:
    
    def __init__(self):
        
        # Нужно ли начинать обновление?
        self.do_we_need_update_data_base = False
        
        # Нужно ли обновить рег-файлы?
        self.do_we_need_regs = False
        
        # Нужны ли нам оперативные обновления?
        self.do_we_need_operups = False
        
        # Нужно ли удалить кеш?
        self.do_we_need_remove_cache = False
        
        # Нужен ли отчет?
        self.do_we_need_report = False
        
        # Сохраняем путь к папке Temp, где будут храниться временные файлы (например, логи)
        tmp_directory='Temp'
        self.tmp_path = os.path.join(CUR_DIR, tmp_directory)
    
    # Проверяем наличие флага в соответствующей папке FLAGS.
    # Если флаг есть, то соответствующий модуль еще не выполнен
    def check_flag(self, sftp):

        # Проходимся по содержимому папки FLAGS
        # https://stackoverflow.com/questions/12295551/how-to-list-all-the-folders-and-files-in-the-directory-after-connecting-through
        for entry in sftp.listdir_attr(PATH_TO_FLAG):
            mode = entry.st_mode
            
            # Если флага FlagBase.txt найден, то базы данных еще не установлены
            if (S_ISREG(mode) and entry.filename=='FlagBase.txt'):
                self.do_we_need_update_data_base = True
            # Аналогично для флага FlagReg.txt
            elif (S_ISREG(mode) and entry.filename=='FlagReg.txt'):
                self.do_we_need_regs = True
            # Аналогично для флага FlagOperup.txt
            elif (S_ISREG(mode) and entry.filename=='FlagOperup.txt'):
                self.do_we_need_operups = True
            # Аналогично для флага FlagCash.txt
            elif (S_ISREG(mode) and entry.filename=='FlagCash.txt'):
                self.do_we_need_remove_cache = True
            # Аналогично для флага FlagSisinfo.txt
            elif(S_ISREG(mode) and entry.filename=='FlagSisinfo.txt'):
                self.do_we_need_report = True
    
    # Совместили все проверки внутри одной, что позволяет сократить код
    def check_all(self, sftp):
        self.check_flag(sftp)

In [7]:
# Функция по удалению файлов из некоторой папки
def delete_files_from_directory(path_to_directory):
    
    # Находим список файлов в заданной папке
    dir_files = [f for f in os.listdir(path_to_directory) if os.path.isfile(os.path.join(path_to_directory, f))]
    
    # Удаляем файлы из папки 
    for f in dir_files:
        os.remove(os.path.join(path_to_directory,f))

### 2. Скачивание баз данных

#### 2.1. Функция скачивания одного файла с сервера

In [18]:
# Функция закачки файла
#
# На вход подается название скачиваемого файла и папка на сервере, куда нужно перейти
# 
# На выходе получаем скачанный файл. 

def download_one_file(download_file, folder_in_server):

    # Локальный путь для загруженного с сервера файла
    local_file_name = os.path.join(PATH_TO_DOWNLOAD,download_file)

    # Поскольку мы находимся в папке с файлом, то теперь удаленный путь к файлу - это имя файла
    remote_path = folder_in_server + '/' + download_file
    
    # Скачиваем файл
    sftp.get(remote_path, local_file_name)

    # Поскольку при скачивании файла изменяется его дата модификации, то нужно 
    # заменить новую дату модификации на старую дату.
    utime = sftp.stat(remote_path).st_mtime
    mtime = (datetime.fromtimestamp(utime))
    os.utime(local_file_name, (float(mtime.timestamp()), float(mtime.timestamp())))

#### 2.2. Функция проверки наличия свободного места перед скачиванием

In [19]:
def check_space(files_to_download):
    
    # Задача функции - узнать объем файлов, который нужно докачать для обновления ПКП.
    # Идея алгоритма функции - пройтись по списку скачиваемых файлов и узнать, сколько файлов уже корректно скачаны.
    # Функция, узнав, какие файлы скачаны корректно, а какие - нет, определяет какой объем файлов еще нужно
    # докачать.
    # ПРИМЕЧАНИЕ: в комментах к текущей ячейке слова "корректно скачан" == "полностью скачан"
    
    logger.info("Началась проверка свободного места на диске")
    disk_free_space_needed = 0
    current_disk_space = psutil.disk_usage(PATH_TO_DOWNLOAD).free

    # Заводим массив для хранения данных о корректности файлов
    # Если в i-ом элементе хранится True, то i-ый файл скачан и корректен.
    # Иначе - в i-ом элементе хранится False.
    are_files_correct = []

    for download_file in files_to_download:

        # Локальный путь для загруженного с сервера файла
        local_file_name = os.path.join(PATH_TO_DOWNLOAD, download_file.attrib['Name'])

        # Вначале проверяем - скачан ли уже файл или нет
        if ((os.path.isfile(local_file_name) == True)):

            # Если файл скачан полностью, то заносим его в массив как корректно скачанный файл
            if (md5(local_file_name).upper() == download_file.attrib['md5']):
                are_files_correct.append(True)

            # Если файл скачан, но не полностью
            else:
                are_files_correct.append(False)

                # Находим, сколько уже скачано
                already_downloaded = os.path.getsize(local_file_name)

                # Находим сколько нужно скачать, т.е. размер файла из xml
                need_to_download = int(download_file.attrib['Size'])

                # Сколько осталось скачать
                rest_part_of_file = need_to_download - already_downloaded

                # Поскольку файл скачан частично, то нужно прибавлять размер
                # оставшейся части файла для скачивания
                disk_free_space_needed += rest_part_of_file

        # Если скачивание файла еще не началось
        else:
            are_files_correct.append(False)

            # Поскольку файл еще не начал скачиваться, то прибавляем весь его размер
            disk_free_space_needed += int(download_file.attrib['Size'])
    
    # Делаем зазор в размере 100 байт между необходимой для обновления баз данных памятью (disk_free_space_needed)
    # и текущим свободным пространством (current_disk_space)
    if (disk_free_space_needed > (current_disk_space + 100)):
        mes1 = "Не хватает места на диске!"
        mes2 = "Необходимо {:.2f}Гб, но доступно только {:.2f}Гб".format((disk_free_space_needed / 10**9), 
                                                                     (current_disk_space / 10**9))
        mes = mes1 + '\n' + mes2
        logger.error(mes)
        
        # Отправляем лог-файл на сервер
        old_logs_path = os.path.join("Temp", LOG_NAME)
        new_logs_path = PATH_TO_SERVER_LOGS + LOG_NAME
        sftp.put(old_logs_path, new_logs_path)
        sys.exit(1)
    
    else:
        mes1 = "На диске достаточно свободного места (Необходимо - {:.1f}Гб, Доступно - {:.1f}Гб)".format((disk_free_space_needed / 10**9), (current_disk_space / 10**9))
        
        mes2 = "Начинается скачивание файлов..."
        mes = mes1 + ' ' + mes2
        logger.info(mes)
        
        return are_files_correct

#### 2.3. Заключительная функция проверки, сверяющая хеши скачанных файлов с хешами из xml

In [20]:
def check_md5(path_to_files, files_to_download):
    
    logger.info("Началась заключительная проверка файлов по md5")
    
    # Проходимся в цикле по списку скачиваемых файлов и проверяем их md5
    for pos, download_file in enumerate(files_to_download):

        # извлекаем имя файла
        download_file_name = download_file.attrib['Name']
        
        full_name = os.path.join(path_to_files, download_file_name)

        # Если файл корректный, то идем дальше
        # Иначе - записываем ошибку в лог-файл и завершаем работу программы
        if (md5(full_name).upper() == download_file.attrib['md5']):
            pass
        else:
            mes = "Md5 файла {} не совпадает с md5 в xml-файле!".format(download_file_name)
            logger.error(mes)
            
            old_logs_path = os.path.join("Temp", LOG_NAME)
            new_logs_path = PATH_TO_SERVER_LOGS + LOG_NAME
            sftp.put(old_logs_path, new_logs_path)
            sys.exit(1)
    
    logger.info("Заключительная проверка файлов по md5 успешно завершена!")

#### 2.4. Функция загрузки всех файлов (баз данных) с сервера

In [21]:
def download_all_files(files_to_download, are_files_correct):
    # Проходимся в цикле по списку скачиваемых файлов и скачиваем их
    for pos, download_file in enumerate(files_to_download):
        
        # Извлекаем имя скачиваемого файла
        download_file_name = download_file.attrib['Name']
        
        mes = "Начинаем скачивать файл {}!".format(download_file_name)
        logger.info(mes)

        # Проверяем, скачан ли файл. Если файл скачан успешно, то просто переходим к следующему файлу
        if (are_files_correct[pos] == True):
            mes = "Файл {} скачан успешно!".format(download_file_name)
            logger.info(mes)
            continue
        else:
            pass

        # Скачиваем файл
        download_one_file(download_file_name, SERVER_BASE_FOLDER)

        # Заводим переменную "состояния" для каджого файла.
        # По умолчанию считаем, что текущий файл некорректен.
        # Если после проверки файл окажется корректным, то изменим значение переменной "состояния"
        is_file_correct = False


        # Будем перезакачивать файл до тех пор, пока он не станет корректным. 
        while (is_file_correct == False):
            
            full_name = os.path.join(PATH_TO_DOWNLOAD, download_file_name)

            # Если файл корректный, то изменяем переменную "состояния"
            if (md5(full_name).upper() == download_file.attrib['md5']):
                is_file_correct = True

            # Если файл некорректный, то закачиваем файл заново
            else:

                # Докачиваем файл
                download_one_file(download_file_name, SERVER_BASE_FOLDER)

        mes = "Файл {} скачан успешно!".format(download_file_name)
        logger.info(mes)
    
    logger.info("Скачивание файлов завершено! Начинаем заключительную проверку файлов по md5 ...")       

### 3. Обновление рег.файлов

In [ ]:
# При наличии флага производим скачивание 
def work_with_regs():
    
    # Если папка RegOld еще не создана в корне Техэксперта, то создаем ее там.
    if (os.path.isdir(os.path.join(PATH_TO_TEXPERT, "RegOld")) == False):
        os.mkdir(os.path.join(PATH_TO_TEXPERT, "RegOld"))
    
    # Находим старый рег и помещаем его в папку RegOld
    # https://stackoverflow.com/questions/3964681/find-all-files-in-a-directory-with-extension-txt-in-python
    for file in os.listdir(PATH_TO_TEXPERT):
        if file.endswith(".reg"):
            # Путь к старому регу до его перемещения
            cur_path_to_old_reg = os.path.join(PATH_TO_TEXPERT, file)
            
            # Путь к старому регу после его перемещения
            future_path_to_old_reg = os.path.join(PATH_TO_TEXPERT, os.path.join("RegOld", file))
            
            # Осуществляем именно копирование + удаление вместо обычного перемещения shutil.move, 
            # поскольку shutil.move не перезатирает файлы с одним и тем же именем
            # Вначале копируем старый рег
            shutil.copy2(cur_path_to_old_reg, future_path_to_old_reg)
            
            # Удаляем старый рег
            os.remove(cur_path_to_old_reg)
            break
    
    # Скачиваем файл RegList.txt  в папку Temp
    reglist_path = os.path.join(XML_PATH, "RegList.txt")
    sftp.get(reglist_path, "Temp/RegList.txt")
    
    # Считываем названия регов из списка и скачиваем их
    with open("Temp/RegList.txt") as reglist:
        regs = reglist.readlines()
        for reg in regs:
            # Избавляемся от символа \n в строке reg
            reg = reg.strip()
            
            reg_path = os.path.join('/' + CLIENT_NAME.upper() + '/REG/', reg)
            sftp.get(reg_path, os.path.join(PATH_TO_TEXPERT, reg))

### 4. Оперативные обновления

In [ ]:
def update_operup():
    
    # Перед закачкой обновлений очищаем каталог operup
    delete_files_from_directory(PATH_TO_OPERUP_DESTINATION)
    
    # Скачиваем список оперативных обновлений в папку Temp
    sftp.get(PATH_TO_SERVER_OPERUP_LIST, "Temp/OperupList.txt")
    
    # Начинаем скачивание оперативных обновлений
    with open("Temp/OperupList.txt") as operup_file:
        operups = operup_file.readlines()
        for operup in operups:
            operup = operup.strip()
            
            logging.info("Началось закачивание оперативного обновления {}".format(operup))
            
            # Откуда скачиваем обновление operup
            cur_path_to_operup = PATH_TO_SERVER_OPERUP + '/' + operup
            
            # Куда скачиваем обновление operup
            future_path_to_operup = os.path.join(PATH_TO_OPERUP_DESTINATION, operup)
            
            sftp.get(cur_path_to_operup, future_path_to_operup)
            
            logging.info("Закачивание оперативного обновления {} завершено успешно!".format(operup))

### 5. Удаление кеша

In [ ]:
def remove_cache():
    for file in os.listdir(PATH_TO_CACHE):
        if file.endswith(".dat"):
            
            # Путь к кеш-файлу
            path_to_dat = os.path.join(PATH_TO_CACHE, file)
            
            os.remove(path_to_dat)

### 6. Запуск/остановка служб

In [22]:
# Функция запуска службы. Если служба запущена успешно, то возвращаем True
# Иначе - False
# Как правило, если функция успешно завершилась, возвращается 0.  (https://stackoverflow.com/questions/1696998/what-is-the-return-value-of-subprocess-call)
# TODO: утверждение про возврат нуля нужно еще проверить
def start_service():
    
    logging.info("Начинается запуск службы.")
    if (sys.platform == 'linux'):
        # service - это имя службы Техэксперта на linux
        p = subprocess.Popen(["systemctl", "start",  service], stdout=subprocess.PIPE)
        stdout, stderr = p.communicate()
        
        if(p.poll() == 0):
            logging.info("Служба запущена успешно!")
        else:
            logging.error("Не удалось запустить службу!")
            # Отправляем файл с логами в соответствующую папку на сервере
            old_logs_path = os.path.join("Temp", LOG_NAME)
            new_logs_path = PATH_TO_SERVER_LOGS + LOG_NAME
            sftp.put(old_logs_path, new_logs_path)
            sys.exit(1)
    else:
        result = subprocess.call('powershell Start-Service "TSERVER64*"', shell=True)
        
        if (result == 0):
            logging.info("Служба запущена успешно!")
        else:
            logging.error("Не удалось запустить службу!")
            # Отправляем файл с логами в соответствующую папку на сервере
            old_logs_path = os.path.join("Temp", LOG_NAME)
            new_logs_path = PATH_TO_SERVER_LOGS + LOG_NAME
            sftp.put(old_logs_path, new_logs_path)
            sys.exit(1)

In [23]:
# Функция остановка службы. Если служба остановлена успешно, то возвращаем True
# Иначе - False
# Как правило, если функция успешно завершилась, возвращается 0.  (https://stackoverflow.com/questions/1696998/what-is-the-return-value-of-subprocess-call)
# TODO: утверждение про возврат нуля нужно еще проверить
def stop_service():
    
    logging.info("Начинается остановка запуск службы.")
    if (sys.platform == 'linux'):
        # service - это имя службы Техэксперта на linux
        p = subprocess.Popen(["systemctl", "stop",  service], stdout=subprocess.PIPE)
        stdout, stderr = p.communicate()
        
        if(p.poll() == 0):
            logging.info("Служба остановлена успешно!")
        else:
            logging.error("Не удалось остановить службу!")
            
            # Отправляем файл с логами в соответствующую папку на сервере
            old_logs_path = os.path.join("Temp", LOG_NAME)
            new_logs_path = PATH_TO_SERVER_LOGS + LOG_NAME
            sftp.put(old_logs_path, new_logs_path)
            sys.exit(1)
    else:
        result = subprocess.call('powershell Stop-Service "TSERVER64*"', shell=True)
        
        if (result == 0):
            logging.info("Служба остановлена успешно!")
            return True
        else:
            logging.error("Не удалось остановить службу!")
            
            # Отправляем файл с логами в соответствующую папку на сервере
            old_logs_path = os.path.join("Temp", LOG_NAME)
            new_logs_path = PATH_TO_SERVER_LOGS + LOG_NAME
            sftp.put(old_logs_path, new_logs_path)
            sys.exit(1)

### 7. Формирование файла-отчета

In [24]:
# TODO: 
# 1. Файл-отчет нужно выкачивать в папку Temp, которую нужно создать в начале работы скрипта
# 2. Очищать папку Temp после окончания работы скрипта
def create_file_report(destination=FILE_REPORT_DESTINATION):
    
    logger.info("Начинается формирование файла отчета")
    
    # Задаем user-agents для python-скрипта, чтобы тот мог подсоединиться к ftp-серверу
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    
    # Скачиваем файл-отчет с сервера
    data = requests.get(FILE_REPORT_URL, headers=headers)
    
    # https://stackoverflow.com/questions/31804799/how-to-get-pdf-filename-with-python-requests
    # Находим имя файла-отчета, которое сервер присвоил этому файлу
    local_file = data.headers.get("content-disposition").split("filename=")[1]

    # https://stackoverflow.com/questions/40950791/remove-quotes-from-string-in-python
    local_file = local_file.replace('"', '')
    
    # Сохраняем этот файл в текущей папке
    with open(local_file, 'wb')as file:
        file.write(data.content)
    
    # Формируем полный путь к файлу-отчету
    destination += local_file
    
    # Отправляем файл-отчет на сервер
    sftp.put(local_file, destination)
    
    # Удаляем скачанный файл-отчет
    os.remove(local_file)
    
    logger.info("Файл отчета сформирован и отправлен на сервер!")

### Основной код

In [ ]:
def main_cache():
    # Если нужно удалить кеш, то удаляем его
    if (checker.do_we_need_remove_cache == True):

        logger.info("Начинается очистка кеша")

        # Останавливаем службу перед заменой регов
        stop_service()

        # Удаляем кеш
        remove_cache()

        # Включаем службу
        start_service()

        # Удаляем флаг кеша
        cache_flag = PATH_TO_FLAG + '/FlagCash.txt'
        sftp.remove(cache_flag)

        logger.info("Очистка кеша завершена успешно!")

In [ ]:
def main_regs():
    # Если нужно обновить рег.файлы, то обновляем
    if (checker.do_we_need_regs == True):

        logger.info("Начинается установка регистрационных файлов")

        # Останавливаем службу перед заменой регов
        stop_service()

        # Заменяем реги
        work_with_regs()

        # Включаем службу
        start_service()

        # Удаляем флаг рег.обновлений
        reg_flag = PATH_TO_FLAG + '/FlagReg.txt'
        sftp.remove(reg_flag)

        logger.info("Установка регистрационных файлов завершена успешно!")

In [ ]:
def main_operups():
    # Если нужны оперативные обновления, то скачиваем их и производим обновление
    if (checker.do_we_need_operups == True):

        logger.info("Начинается установка оперативных обновлений")

        # Останавливаем службу перед заменой регов
        stop_service()

        # Производим скачивание оперативных обновлений
        update_operup()

        # Включаем службу
        start_service()

        # Удаляем флаг оперативных обновлений
        operup_flag = PATH_TO_FLAG + '/FlagOperup.txt'
        sftp.remove(operup_flag)

        logger.info("Установка оперативных обновлений завершена успешно!")

In [ ]:
def main_bd():
    # Если нужно скачать базы данных, то скачиваем их
    if (checker.do_we_need_update_data_base == True):

        logger.info("Начинается скачивание и установка баз данных")

        # Останавливаем службу перед копированием 
        stop_service()

        # Скачиваем xml-файл со списком закачиваемых томов и их характеристиками
        download_one_file(XML_NAME, XML_PATH)

        # Формируем путь к xml-файлу
        xml = os.path.join(PATH_TO_DOWNLOAD, XML_NAME)

        tree = ET.parse(xml)
        files_to_download = tree.getroot()

        # Проверяем наличие свободного пространства для закачки
        are_files_correct = check_space(files_to_download)

        # Начинаем закачку/докачку файлов с сервера
        download_all_files(files_to_download, are_files_correct)

        # Заключительная проверка уже скачанных файлов
        check_md5(PATH_TO_DOWNLOAD, files_to_download)

        # Включаем службу после успешного копирования
        start_service()

        # Удаляем флаг обновлений баз данных
        base_flag = PATH_TO_FLAG + '/FlagBase.txt'
        sftp.remove(base_flag)

        logger.info("Скачивание и установка баз данных завершены успешно!")

In [ ]:
def main_report():
    # Если отчет еще не отправлен
    if (checker.do_we_need_report == True):
        create_file_report()

        # Удаляем флаг отчета баз данных
        report_flag = PATH_TO_FLAG + '/FlagSisinfo.txt'
        sftp.remove(report_flag)

In [ ]:
def main():    
    # Выполнение основных операций
    main_cache()
    main_regs()
    main_operups()
    main_bd()
    main_report()

In [ ]:
try:
    # Перед началом работы удаляем все файлы из папки Temp
    delete_files_from_directory('Temp')
    
    ############################################### Формируем лог-файл ###############################################

    # Формируем имя для лога
    dt = datetime.today()
    year = str(dt.year)
    month = '{:02d}'.format(dt.month)
    day = '{:02d}'.format(dt.day)
    hour = '{:02d}'.format(dt.hour)
    minute = '{:02d}'.format(dt.minute)

    LOG_NAME = "Logs_{}_{}_{}_{}_{}.txt".format(year, month, day, hour, minute)

    # Задаем конфигурации для формирования файла логов
    # Про конфигурации:
    # https://stackoverflow.com/questions/6386698/how-to-write-to-a-file-using-the-logging-python-module
    # https://docs.python.org/2/howto/logging.html#logging-basic-tutorial
    logging.basicConfig(filename=os.path.join(PATH_TO_LOG,LOG_NAME),
                        filemode = 'a',
                        level=logging.DEBUG,
                        format='%(asctime)s %(levelname)s %(message)s', 
                        datefmt='%d/%m/%Y %H:%M:%S')

    logger = logging.getLogger("Logs")
    
    logger.info("Начало работы программы")
    
    # Создаем объект класса Checker
    checker = Checker()

    # Проверяем, установлены ли базы данных
    checker.check_all(sftp)
    
    # Исполняем основной код
    main()
    
    # Записываем в лог информацию о конце работы программы
    logger.info("Конец работы программы")

    # Закрываем файл с логами
    logging.shutdown()

    # Отправляем файл с логами в соответствующую папку на сервере
    old_logs_path = os.path.join("Temp", LOG_NAME)
    new_logs_path = PATH_TO_SERVER_LOGS + LOG_NAME
    sftp.put(old_logs_path, new_logs_path)

    # Закрываем канал связи
    transport.close()
    
    # Удаляем все файлы из папки Temp
    delete_files_from_directory('Temp')
    
except Exception as e:
    
    logging.exception(e)
    
    # Формируем лог и отправляем его на сервер
    old_logs_path = os.path.join("Temp", LOG_NAME)
    new_logs_path = PATH_TO_SERVER_LOGS + LOG_NAME
    sftp.put(old_logs_path, new_logs_path)